<a href="https://colab.research.google.com/github/riddhisahni/SpeechEmotionRecognition/blob/main/EmotionRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Installations and Imports

In [2]:
!pip install kagglehub
!pip install torchaudio transformers librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [3]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
pip install audiomentations==0.22.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.11.0
    Uninstalling librosa-0.11.0:
      Successfully uninstalled librosa-0.11.0


In [5]:
import kagglehub
import os
import librosa
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification
import torch
import pandas as pd
from datasets import Dataset
from transformers import Wav2Vec2Processor, Trainer, TrainingArguments, Wav2Vec2Config
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Gain

In [6]:
# Download latest version of the dataset
path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")
print("Dataset downloaded to:", path)

#Find all .wav files
all_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".wav"):
            all_files.append(os.path.join(root, file))

print(f"Found {len(all_files)} audio files.")

Dataset downloaded to: /kaggle/input/ravdess-emotional-speech-audio
Found 2880 audio files.


### Data Preparation

In [7]:
# Loop through all files and get emotion from label
data = []
code_to_emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}


def get_true_label_from_filename(filename):
    emotion_code = filename.split("-")[2]  # 3rd field (0-indexed)
    return code_to_emotion.get(emotion_code, "unknown")

for file in all_files:
    fname = os.path.basename(file)
    true = get_true_label_from_filename(fname)
    data.append({"file_path": file, "labeled_emotion": true})

df = pd.DataFrame(data)
print(df.head(100))

                                            file_path labeled_emotion
0   /kaggle/input/ravdess-emotional-speech-audio/A...       surprised
1   /kaggle/input/ravdess-emotional-speech-audio/A...         neutral
2   /kaggle/input/ravdess-emotional-speech-audio/A...         disgust
3   /kaggle/input/ravdess-emotional-speech-audio/A...         disgust
4   /kaggle/input/ravdess-emotional-speech-audio/A...         neutral
..                                                ...             ...
95  /kaggle/input/ravdess-emotional-speech-audio/A...         fearful
96  /kaggle/input/ravdess-emotional-speech-audio/A...            calm
97  /kaggle/input/ravdess-emotional-speech-audio/A...           angry
98  /kaggle/input/ravdess-emotional-speech-audio/A...         fearful
99  /kaggle/input/ravdess-emotional-speech-audio/A...       surprised

[100 rows x 2 columns]


In [8]:
#Create mappings from the list of emotions - NOT SURE EXACTLY HOW WE'RE USING THIS
label_list = df['labeled_emotion'].unique().tolist()

# Create mappings for label2id and id2label
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

print("label2id:", label2id)
print("id2label:", id2label)


label2id: {'surprised': 0, 'neutral': 1, 'disgust': 2, 'fearful': 3, 'sad': 4, 'calm': 5, 'happy': 6, 'angry': 7}
id2label: {0: 'surprised', 1: 'neutral', 2: 'disgust', 3: 'fearful', 4: 'sad', 5: 'calm', 6: 'happy', 7: 'angry'}


In [9]:
#TRYING TO FIX ACCURACY

In [10]:
# Define the augmentation pipeline
def augment_audio(audio, sample_rate=16000):
    augmenter = Compose([
        # Add random Gaussian noise
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),

        # Time Stretching (speed change)
        TimeStretch(min_rate=0.8, max_rate=1.2, p=0.5),

        # Pitch Shifting (to simulate change in pitch)
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5),

        # Randomly shift audio in time
        Shift(min_fraction=-0.1, max_fraction=0.1, p=0.5),

        # Random volume change
        Gain(min_gain_in_db=-6, max_gain_in_db=6, p=0.5)
    ])

    # Apply the augmentations to the audio
    augmented_audio = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented_audio


In [11]:
def preprocess_function(batch):
    # Load the audio from file
    file_path = batch["file_path"]
    speech_array, sampling_rate = librosa.load(file_path, sr=8000)  # Load at 16kHz

    # Apply augmentation (if it's for training data)
    if 'train' in batch:
        speech_array = augmentation_pipeline(samples=speech_array, sample_rate=sampling_rate)

    # Extract features using the feature extractor (without using the processor)
    inputs = feature_extractor(
        speech_array,
        sampling_rate=sampling_rate,
        padding=True,
        return_tensors="pt"
    )

    # Return the input values and the label
    return {
        "input_values": inputs.input_values[0],  # The extracted input values
        "labels": label2id[batch["labeled_emotion"]]  # Label ID
    }


### Model Setup

In [12]:
#Prepare the feature extractor
model_name = "facebook/wav2vec2-base" #IS THIS THE RIGHT MODEL?

# Load feature extractor and model
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name, sampling_rate=8000)

#Freeze the feature extractor - POTENTIALLY CHANGE
model.freeze_feature_extractor()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2306: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [13]:
#Create test/train split

df['emotion'] = df['labeled_emotion'].map(label2id)
dataset = Dataset.from_pandas(df)

split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

# Apply the preprocessing function (including augmentation) to the dataset
train_dataset = train_dataset.map(preprocess_function, remove_columns=["file_path", "labeled_emotion"])
test_dataset = test_dataset.map(preprocess_function, remove_columns=["file_path", "labeled_emotion"])


Map:   0%|          | 0/2304 [00:00<?, ? examples/s]

Map:   0%|          | 0/576 [00:00<?, ? examples/s]

In [14]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 2304
Test dataset size: 576


In [15]:
'''
Dropout: You may want to adjust the dropout values based on your task and experiment with different configurations.
Freezing: If you have frozen the feature extractor (as you did earlier), it may not be necessary to adjust dropout, but if you are fine-tuning the whole model, keep dropout to avoid overfitting.
'''

config = Wav2Vec2Config.from_pretrained(
    model_name,
    num_labels=len(label_list),
    label2id=label2id,
    id2label=id2label,
    hidden_dropout=0.3,       # dropout in transformer layers
    attention_dropout=0.3,    # dropout in attention
)

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name,
    config=config
)

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="./wav2vec2_finetuned_emotion_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)


In [17]:
# Define compute_metrics function
def compute_metrics(p):
    predictions, labels = p
    # Convert logits to predicted class indices (this will give you the predicted label for each sample)
    preds = predictions.argmax(axis=1)
    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

In [18]:
# Step 7: Initialize the Trainer
data_collator = DataCollatorWithPadding(feature_extractor)
trainer = Trainer(
    model=model,  # The model you're fine-tuning
    args=training_args,  # Training arguments
    train_dataset=train_dataset,  # Your preprocessed training dataset
    eval_dataset=test_dataset,  # You can use a separate validation dataset if you have one
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Pass the compute_metrics function here

)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.937600,1.799126,0.296875
2,1.675100,1.688595,0.359375
3,1.416900,1.211290,0.593750
4,0.968500,1.181862,0.585069
5,0.879800,1.142772,0.645833
6,0.685200,0.569481,0.814236
7,0.575900,0.535026,0.838542
8,0.397200,0.835806,0.784722
9,0.493200,0.524759,0.873264
10,0.343800,0.382640,0.901042


In [ ]:
# After training is complete, save the model and feature extractor
model.save_pretrained("./finetuned_wav2vec2_emotion_model")
feature_extractor.save_pretrained("./finetuned_wav2vec2_emotion_model")

# Check the saved files
import os
print(os.listdir("./finetuned_wav2vec2_emotion_model"))


In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
# Add these imports to your notebook
!pip install ipywidgets sounddevice scipy

In [ ]:
# Install necessary packages
!pip install ipywidgets

# Import required libraries
import os
import numpy as np
import torch
import librosa
import IPython.display as ipd
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Label, Output, FileUpload
from google.colab import files  # For Google Colab
import pandas as pd
import matplotlib.pyplot as plt
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification

# Define class for file-based emotion detection
class EmotionDetectorFileUpload:
    def __init__(self, model, feature_extractor, id2label):
        self.model = model
        self.feature_extractor = feature_extractor
        self.id2label = id2label
        self.output = Output()
        self.history = []  # To store detection history
        self.setup_ui()

    def setup_ui(self):
        # Create UI components
        self.upload_button = FileUpload(
            accept='.wav',  # Accept only .wav files
            multiple=False,  # Allow only one file
            description='Select Audio File'
        )
        self.upload_button.observe(self.handle_upload, names='value')

        self.status_label = Label(value="Upload an audio file to detect emotion")
        self.result_label = Label(value="No emotion detected yet")

        # Add buttons for batch operations
        self.batch_upload_btn = Button(description="Upload Multiple Files")
        self.batch_upload_btn.on_click(self.batch_upload)

        # Add a section to show detection history
        self.history_output = Output()

        # Layout
        self.ui = VBox([
            Label(value="Speech Emotion Detector"),
            self.status_label,
            HBox([self.upload_button, self.batch_upload_btn]),
            self.result_label,
            self.output,
            Label(value="Detection History:"),
            self.history_output
        ])

        # Display the UI
        display(self.ui)

    def handle_upload(self, change):
        """Handle file upload event"""
        if not change.new:
            return

        # Get uploaded file
        for name, file_info in change.new.items():
            self.status_label.value = f"Processing {name}..."

            # Save the file temporarily
            with open(name, 'wb') as f:
                f.write(file_info['content'])

            # Display audio player
            with self.output:
                self.output.clear_output()
                display(ipd.Audio(name, autoplay=False))

            # Process the file
            self.detect_emotion(name)

            # Clear upload widget for next use
            self.upload_button.value.clear()
            break

    def detect_emotion(self, audio_file):
        """Detect emotion in the given audio file"""
        try:
            # Load and process audio
            speech_array, sampling_rate = librosa.load(audio_file, sr=8000)

            # Extract features
            inputs = self.feature_extractor(
                speech_array,
                sampling_rate=sampling_rate,
                padding=True,
                return_tensors="pt"
            )

            # Make sure inputs are on the same device as the model
            device = next(self.model.parameters()).device
            input_values = inputs.input_values.to(device)

            # Get prediction
            with torch.no_grad():
                outputs = self.model(input_values)
                logits = outputs.logits
                predicted_class_id = torch.argmax(logits, dim=1).item()

                # Get confidence scores for all emotions
                confidences = torch.softmax(logits, dim=1)[0].tolist()
                confidence = confidences[predicted_class_id]
                predicted_emotion = self.id2label[predicted_class_id]

            # Update UI
            self.status_label.value = "Ready for next file"
            self.result_label.value = f"Detected Emotion: {predicted_emotion.capitalize()} (Confidence: {confidence:.2f})"

            # Add to history
            self.history.append({
                'file': os.path.basename(audio_file),
                'emotion': predicted_emotion,
                'confidence': confidence,
                'all_confidences': {self.id2label[i]: conf for i, conf in enumerate(confidences)}
            })

            # Update history display
            self.update_history_display()

            return predicted_emotion, confidence

        except Exception as e:
            self.status_label.value = f"Error: {str(e)}"
            print(f"Error processing audio: {e}")
            return None, 0

    def batch_upload(self, button):
        """Handle batch upload for multiple files"""
        self.status_label.value = "Upload multiple .wav files..."

        try:
            # For Google Colab
            uploaded = files.upload()

            if not uploaded:
                self.status_label.value = "No files uploaded"
                return

            results = []

            for filename, content in uploaded.items():
                if filename.endswith('.wav'):
                    # Save the file temporarily
                    with open(filename, 'wb') as f:
                        f.write(content)

                    # Process file
                    emotion, confidence = self.detect_emotion(filename)
                    if emotion:
                        results.append((filename, emotion, confidence))

            # Show summary
            self.status_label.value = f"Processed {len(results)} files"

        except Exception as e:
            self.status_label.value = f"Error in batch processing: {str(e)}"

    def update_history_display(self):
        """Update the history display with latest results"""
        with self.history_output:
            self.history_output.clear_output()

            if not self.history:
                print("No detection history yet")
                return

            # Create a DataFrame for better display
            history_df = pd.DataFrame([
                {'File': item['file'],
                 'Emotion': item['emotion'],
                 'Confidence': f"{item['confidence']:.2f}"}
                for item in self.history
            ])

            display(history_df)

            # Also show a visualization of the most recent result
            if self.history:
                latest = self.history[-1]
                emotions = list(latest['all_confidences'].keys())
                confidences = list(latest['all_confidences'].values())

                plt.figure(figsize=(10, 4))
                bars = plt.bar(emotions, confidences)
                plt.ylabel('Confidence')
                plt.title(f'Emotion Confidence Scores for {latest["file"]}')
                plt.xticks(rotation=45)

                # Highlight the detected emotion
                for i, emotion in enumerate(emotions):
                    if emotion == latest['emotion']:
                        bars[i].set_color('red')

                plt.tight_layout()
                plt.show()

    def get_stats(self):
        """Get statistics on detected emotions"""
        if not self.history:
            return "No detection history available"

        emotion_counts = {}
        for item in self.history:
            emotion = item['emotion']
            emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

        # Display as pie chart
        labels = list(emotion_counts.keys())
        sizes = list(emotion_counts.values())

        plt.figure(figsize=(8, 8))
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
        plt.axis('equal')
        plt.title('Distribution of Detected Emotions')
        plt.show()

        return pd.DataFrame({'Emotion': labels, 'Count': sizes})

# Add this code to the end of your notebook to use the file upload detector

# First, ensure your model is loaded (use the same code from your training)
# If you haven't saved the model yet, add this code after trainer.train():
model_path = "./wav2vec2_finetuned_emotion_model"

# Load the model and feature extractor
try:
    # Try to load the saved model
    model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base", sampling_rate=8000)

    # Make sure model is on CPU if CUDA is being used but not needed
    if torch.cuda.is_available():
        # Option 1: Move model to CPU if you're having device mismatch issues
        model = model.cpu()
        print("Model moved to CPU to avoid device mismatch")

        # Option 2 (alternative): Move model to GPU if you want GPU acceleration
        # model = model.cuda()
        # print("Model moved to GPU for faster processing")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Using the model from training session if available")
    # If loading fails, use the model from the current session
    # (assuming it's already trained and available in the notebook)

# Create the id2label mapping (use the one from your training code)
id2label = {i: label for i, label in enumerate(label_list)}

# Create and display the detector UI
detector = EmotionDetectorFileUpload(model, feature_extractor, id2label)

# You can also add a function to batch process files from a directory
def process_directory(directory_path):
    """Process all WAV files in a directory"""
    import glob

    # Get all .wav files
    audio_files = glob.glob(os.path.join(directory_path, "*.wav"))

    if not audio_files:
        print(f"No WAV files found in {directory_path}")
        return []

    results = []
    for audio_file in audio_files:
        emotion, confidence = detector.detect_emotion(audio_file)
        if emotion:
            results.append((audio_file, emotion, confidence))

    # Display summary
    print(f"Processed {len(results)} files from {directory_path}")
    summary_df = pd.DataFrame([
        {'File': os.path.basename(r[0]), 'Emotion': r[1], 'Confidence': f"{r[2]:.2f}"}
        for r in results
    ])

    display(summary_df)
    return results

# Function to save batch processing results to CSV
def save_results_to_csv(results, output_file="emotion_results.csv"):
    """Save batch processing results to CSV"""
    if not results:
        print("No results to save")
        return

    df = pd.DataFrame([
        {'File': os.path.basename(r[0]) if isinstance(r[0], str) else r[0],
         'Emotion': r[1],
         'Confidence': r[2]}
        for r in results
    ])

    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    return df

# Example usage:
# results = process_directory("path/to/audio/files")
# save_results_to_csv(results)

# Optimize model for deployment
def optimize_model():
    """Create an optimized version of the model for faster inference"""
    try:
        # Create directory if it doesn't exist
        os.makedirs("optimized_model", exist_ok=True)

        # Save model in optimized format
        model.save_pretrained("optimized_model")
        feature_extractor.save_pretrained("optimized_model")

        # Create a config file with label mappings
        with open("optimized_model/id2label.json", "w") as f:
            import json
            json.dump(id2label, f)

        print("Model optimized and saved to 'optimized_model' directory")
    except Exception as e:
        print(f"Error optimizing model: {e}")

# Uncomment to optimize your model
# optimize_model()

# Add a method to analyze an entire dataset and visualize results
def analyze_dataset(dataset_path):
    """Analyze an entire dataset of audio files and show emotion distribution"""
    results = process_directory(dataset_path)

    if not results:
        return

    # Get emotion counts
    emotion_counts = {}
    for _, emotion, _ in results:
        emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

    # Plot distribution
    labels = list(emotion_counts.keys())
    sizes = list(emotion_counts.values())

    plt.figure(figsize=(12, 6))

    # Bar chart
    plt.subplot(1, 2, 1)
    plt.bar(labels, sizes)
    plt.xlabel('Emotion')
    plt.ylabel('Count')
    plt.title('Emotion Distribution')
    plt.xticks(rotation=45)

    # Pie chart
    plt.subplot(1, 2, 2)
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    plt.axis('equal')
    plt.title('Percentage Distribution')

    plt.tight_layout()
    plt.show()

    return pd.DataFrame({'Emotion': labels, 'Count': sizes})

# Example usage:
# emotion_stats = analyze_dataset("path/to/dataset")